## Intégrations

Dans notre exemple précédent, nous avons travaillé avec des vecteurs bag-of-words de haute dimension de longueur `vocab_size`, et nous avons explicitement converti des vecteurs de représentation positionnelle de basse dimension en une représentation clairsemée à un seul bit actif. Cette représentation à un seul bit actif n'est pas efficace en termes de mémoire. De plus, chaque mot est traité indépendamment des autres, ce qui fait que les vecteurs encodés de cette manière ne reflètent pas les similitudes sémantiques entre les mots.

Dans cette unité, nous continuerons à explorer le dataset **News AG**. Pour commencer, chargeons les données et récupérons quelques définitions de l'unité précédente.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Qu'est-ce qu'un embedding ?

L'idée d'un **embedding** est de représenter les mots à l'aide de vecteurs denses de dimension inférieure qui reflètent le sens sémantique du mot. Nous verrons plus tard comment construire des embeddings de mots significatifs, mais pour l'instant, considérons simplement les embeddings comme un moyen de réduire la dimensionnalité d'un vecteur de mots.

Ainsi, une couche d'embedding prend un mot en entrée et produit un vecteur de sortie de taille `embedding_size`. En un sens, cela ressemble beaucoup à une couche `Dense`, mais au lieu de prendre un vecteur one-hot encodé en entrée, elle peut prendre un numéro de mot.

En utilisant une couche d'embedding comme première couche de notre réseau, nous pouvons passer d'un modèle bag-of-words à un modèle **embedding bag**, où nous convertissons d'abord chaque mot de notre texte en l'embedding correspondant, puis calculons une fonction d'agrégation sur tous ces embeddings, comme `sum`, `average` ou `max`.

![Image montrant un classificateur avec embedding pour cinq mots de séquence.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Notre réseau de neurones classificateur se compose des couches suivantes :

* Une couche `TextVectorization`, qui prend une chaîne de caractères en entrée et produit un tenseur de numéros de tokens. Nous spécifierons une taille de vocabulaire raisonnable `vocab_size` et ignorerons les mots moins fréquemment utilisés. La forme d'entrée sera 1, et la forme de sortie sera $n$, car nous obtiendrons $n$ tokens en résultat, chacun contenant des numéros allant de 0 à `vocab_size`.
* Une couche `Embedding`, qui prend $n$ numéros et réduit chaque numéro à un vecteur dense d'une longueur donnée (100 dans notre exemple). Ainsi, le tenseur d'entrée de forme $n$ sera transformé en un tenseur de forme $n\times 100$.
* Une couche d'agrégation, qui calcule la moyenne de ce tenseur le long du premier axe, c'est-à-dire qu'elle calculera la moyenne de tous les $n$ tenseurs d'entrée correspondant à différents mots. Pour implémenter cette couche, nous utiliserons une couche `Lambda` et lui passerons la fonction pour calculer la moyenne. La sortie aura une forme de 100, et ce sera la représentation numérique de toute la séquence d'entrée.
* Enfin, un classificateur linéaire `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Dans le résumé, dans la colonne **forme de sortie**, la première dimension du tenseur `None` correspond à la taille du lot (minibatch), et la seconde correspond à la longueur de la séquence de tokens. Toutes les séquences de tokens dans le lot ont des longueurs différentes. Nous verrons comment gérer cela dans la section suivante.

Passons maintenant à l'entraînement du réseau :


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Note** que nous construisons un vectoriseur basé sur un sous-ensemble des données. Cela est fait afin d'accélérer le processus, et cela pourrait entraîner une situation où tous les tokens de notre texte ne sont pas présents dans le vocabulaire. Dans ce cas, ces tokens seraient ignorés, ce qui pourrait entraîner une précision légèrement inférieure. Cependant, dans la réalité, un sous-ensemble de texte donne souvent une bonne estimation du vocabulaire.


### Gestion des tailles de séquences de variables

Comprenons comment l'entraînement se déroule dans les mini-lots. Dans l'exemple ci-dessus, le tenseur d'entrée a une dimension de 1, et nous utilisons des mini-lots de taille 128, ce qui donne une taille réelle du tenseur de $128 \times 1$. Cependant, le nombre de tokens dans chaque phrase est différent. Si nous appliquons la couche `TextVectorization` à une seule entrée, le nombre de tokens retournés varie en fonction de la manière dont le texte est tokenisé :


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Cependant, lorsque nous appliquons le vectoriseur à plusieurs séquences, il doit produire un tenseur de forme rectangulaire, donc il remplit les éléments inutilisés avec le jeton PAD (qui dans notre cas est zéro) :


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Voici les incorporations :


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Remarque** : Pour minimiser la quantité de remplissage, il peut être judicieux dans certains cas de trier toutes les séquences du jeu de données par ordre croissant de longueur (ou, plus précisément, par nombre de tokens). Cela garantira que chaque minibatch contient des séquences de longueur similaire.


## Incrustations sémantiques : Word2Vec

Dans notre exemple précédent, la couche d'incrustation a appris à mapper des mots à des représentations vectorielles, mais ces représentations n'avaient pas de signification sémantique. Il serait intéressant d'apprendre une représentation vectorielle où des mots similaires ou des synonymes correspondent à des vecteurs proches les uns des autres selon une certaine distance vectorielle (par exemple, la distance euclidienne).

Pour cela, nous devons préentraîner notre modèle d'incrustation sur une grande collection de textes en utilisant une technique telle que [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Cette méthode repose sur deux architectures principales utilisées pour produire une représentation distribuée des mots :

 - **Sac de mots continu** (CBoW), où l'on entraîne le modèle à prédire un mot à partir du contexte environnant. Étant donné le ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, l'objectif du modèle est de prédire $W_0$ à partir de $(W_{-2},W_{-1},W_1,W_2)$.
 - **Skip-gram continu**, qui est l'opposé du CBoW. Le modèle utilise la fenêtre de mots du contexte environnant pour prédire le mot actuel.

CBoW est plus rapide, tandis que skip-gram, bien que plus lent, représente mieux les mots peu fréquents.

![Image montrant les algorithmes CBoW et Skip-Gram pour convertir des mots en vecteurs.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Pour expérimenter avec l'incrustation Word2Vec préentraînée sur le dataset Google News, nous pouvons utiliser la bibliothèque **gensim**. Ci-dessous, nous trouvons les mots les plus similaires à 'neural'.

> **Note:** Lorsque vous créez des vecteurs de mots pour la première fois, leur téléchargement peut prendre un certain temps !


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Nous pouvons également extraire l'incorporation vectorielle du mot, à utiliser dans l'entraînement du modèle de classification. L'incorporation comporte 300 composantes, mais ici nous montrons seulement les 20 premières composantes du vecteur pour plus de clarté :


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

La grande particularité des embeddings sémantiques est que vous pouvez manipuler l'encodage vectoriel en fonction des sémantiques. Par exemple, nous pouvons demander de trouver un mot dont la représentation vectorielle est aussi proche que possible des mots *roi* et *femme*, et aussi éloignée que possible du mot *homme* :


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Un exemple ci-dessus utilise une certaine magie interne de GenSym, mais la logique sous-jacente est en réalité assez simple. Une chose intéressante à propos des embeddings est que vous pouvez effectuer des opérations vectorielles normales sur les vecteurs d'embedding, et cela refléterait des opérations sur les **significations** des mots. L'exemple ci-dessus peut être exprimé en termes d'opérations vectorielles : nous calculons le vecteur correspondant à **ROI-HOMME+FEMME** (les opérations `+` et `-` sont effectuées sur les représentations vectorielles des mots correspondants), puis nous trouvons le mot le plus proche dans le dictionnaire de ce vecteur :


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE** : Nous avons dû ajouter de petits coefficients aux vecteurs *homme* et *femme* - essayez de les supprimer pour voir ce qui se passe.

Pour trouver le vecteur le plus proche, nous utilisons les outils de TensorFlow pour calculer un vecteur de distances entre notre vecteur et tous les vecteurs du vocabulaire, puis nous trouvons l'index du mot minimal en utilisant `argmin`.


Bien que Word2Vec semble être un excellent moyen d'exprimer la sémantique des mots, il présente de nombreux inconvénients, notamment les suivants :

* Les modèles CBoW et skip-gram sont des **représentations prédictives**, et ils ne prennent en compte que le contexte local. Word2Vec ne profite pas du contexte global.
* Word2Vec ne prend pas en compte la **morphologie** des mots, c'est-à-dire le fait que le sens d'un mot peut dépendre de différentes parties du mot, comme la racine.

**FastText** tente de surmonter cette deuxième limitation et s'appuie sur Word2Vec en apprenant des représentations vectorielles pour chaque mot ainsi que pour les n-grammes de caractères trouvés dans chaque mot. Les valeurs des représentations sont ensuite moyennées en un seul vecteur à chaque étape d'entraînement. Bien que cela ajoute beaucoup de calculs supplémentaires lors de la pré-formation, cela permet aux représentations vectorielles d'intégrer des informations sur les sous-mots.

Une autre méthode, **GloVe**, utilise une approche différente pour les représentations vectorielles, basée sur la factorisation de la matrice mot-contexte. Tout d'abord, elle construit une grande matrice qui compte le nombre d'occurrences des mots dans différents contextes, puis elle tente de représenter cette matrice dans des dimensions inférieures de manière à minimiser la perte de reconstruction.

La bibliothèque gensim prend en charge ces représentations vectorielles, et vous pouvez les expérimenter en modifiant le code de chargement du modèle ci-dessus.


## Utiliser des embeddings préentraînés dans Keras

Nous pouvons modifier l'exemple ci-dessus pour préremplir la matrice de notre couche d'embedding avec des embeddings sémantiques, tels que Word2Vec. Les vocabulaires de l'embedding préentraîné et du corpus de texte ne correspondront probablement pas, donc nous devons en choisir un. Ici, nous explorons les deux options possibles : utiliser le vocabulaire du tokenizer et utiliser le vocabulaire des embeddings Word2Vec.

### Utiliser le vocabulaire du tokenizer

En utilisant le vocabulaire du tokenizer, certains mots du vocabulaire auront des embeddings Word2Vec correspondants, tandis que d'autres seront absents. Étant donné que la taille de notre vocabulaire est `vocab_size`, et que la longueur du vecteur d'embedding Word2Vec est `embed_size`, la couche d'embedding sera représentée par une matrice de poids de forme `vocab_size`$\times$`embed_size`. Nous remplirons cette matrice en parcourant le vocabulaire :


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Pour les mots qui ne sont pas présents dans le vocabulaire de Word2Vec, nous pouvons soit les laisser comme des zéros, soit générer un vecteur aléatoire.

Nous pouvons maintenant définir une couche d'embedding avec des poids préentraînés :


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Remarque** : Notez que nous avons défini `trainable=False` lors de la création de `Embedding`, ce qui signifie que nous ne réentraînons pas la couche Embedding. Cela peut entraîner une légère baisse de précision, mais cela accélère l'entraînement.

### Utilisation du vocabulaire d'embedding

Un problème avec l'approche précédente est que les vocabulaires utilisés dans TextVectorization et Embedding sont différents. Pour résoudre ce problème, nous pouvons utiliser l'une des solutions suivantes :
* Réentraîner le modèle Word2Vec sur notre vocabulaire.
* Charger notre jeu de données avec le vocabulaire du modèle Word2Vec préentraîné. Les vocabulaires utilisés pour charger le jeu de données peuvent être spécifiés lors du chargement.

La deuxième approche semble plus simple, alors mettons-la en œuvre. Tout d'abord, nous allons créer une couche `TextVectorization` avec le vocabulaire spécifié, tiré des embeddings Word2Vec :


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

La bibliothèque d'embeddings de mots gensim contient une fonction pratique, `get_keras_embeddings`, qui créera automatiquement la couche d'embeddings Keras correspondante pour vous.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Une des raisons pour lesquelles nous n'observons pas une précision plus élevée est que certains mots de notre ensemble de données sont absents du vocabulaire préentraîné de GloVe, et sont donc essentiellement ignorés. Pour surmonter cela, nous pouvons entraîner nos propres embeddings basés sur notre ensemble de données.


## Les embeddings contextuels

Une des principales limites des représentations d'embeddings préentraînés traditionnels comme Word2Vec est qu'ils ne peuvent pas différencier les différents sens d'un mot, même s'ils peuvent en capturer une partie de la signification. Cela peut poser des problèmes dans les modèles en aval.

Par exemple, le mot "play" a des significations différentes dans ces deux phrases :
- Je suis allé voir une **pièce** au théâtre.
- John veut **jouer** avec ses amis.

Les embeddings préentraînés dont nous avons parlé représentent les deux sens du mot "play" dans le même embedding. Pour surmonter cette limitation, nous devons construire des embeddings basés sur le **modèle de langage**, qui est entraîné sur un large corpus de texte et *sait* comment les mots peuvent être assemblés dans différents contextes. Discuter des embeddings contextuels dépasse le cadre de ce tutoriel, mais nous y reviendrons en parlant des modèles de langage dans la prochaine unité.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
